# VQE Basic Estimator

This notebook intends to give a **basic** introduction on the Variational Quantum Eigensolver routine. 

We will infer the **minimal eigenvalue** of the **Pauli-Z matrix by executing the VQE routine on a Quantum Simulator and a real Quantum Computer**

## Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as tck
from qiskit_aer import AerSimulator
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.quantum_info.operators import Operator
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Options, Estimator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_algorithms.optimizers import SPSA
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.visualization import visualize_transition
service = QiskitRuntimeService(channel='ibm_quantum')

In [ ]:
import warnings
warnings.simplefilter(action='ignore')

Some helper functions for plotting.

Nothing fancy here :). 

In [ ]:
def plot_expect_landscape(phases,result_landscape,params=None,energy_list=None):
    plt.plot(phases/np.pi, result_landscape.values, label=r'$E= \langle 0| R_y^{\dagger}(\theta) Z  R_y(\theta) |0 \rangle$')
    if params != None and energy_list != None:
        plt.plot(np.array(params)/np.pi,energy_list,'o',label='Optimizer')
    plt.title(r'Expectation value $<\psi(\theta)|Z|\psi(\theta)>$ with variation in $\theta$')
    ax = plt.gca()
    ax.grid(True)
    ax.xaxis.set_major_formatter(tck.FormatStrFormatter('%g $\pi$'))
    ax.xaxis.set_major_locator(tck.MultipleLocator(base=1/2))
    plt.xlabel(r'$\theta$')
    plt.ylabel(r'$<\psi(\theta)|Z|\psi(\theta)>$')
    plt.legend()
    plt.show()

def backend_information(backend):
    print('Configuration:')
    print('  Name: ', backend.configuration().backend_name)
    print('  Version: ', backend.configuration().backend_version)
    print('  N-Qubits: ', backend.configuration().n_qubits)
    print('  Basis Gates: ',backend.configuration().basis_gates)
    print('Status:')
    print('  Operational: ', backend.status().operational)
    print('  Pending jobs:', backend.status().pending_jobs)
    print('  Status message:', backend.status().status_msg)

## Defining the optimization problem

Lets define a simple toy problem: The Pauli-Z matrix.

In [ ]:
# Define the Pauli Z operator "ops" using the SparsePauliOp class
# Print the operator.

# Your code goes here:


## Defining the ansatz

We choose a parameterized quantum circuit (Rotational-Y-Gate) as ansatz.

By varying the free parameter $\theta$ of this circuit we will derive ever closer approximations to the lowest eigenvalue of our problem.

(live coding)

In [ ]:
# Define the ansatz: a function that creates a QC with a rot-y gate applied to the 1st qubit

# Your code goes here:


In [ ]:
# Define a placeholder parameter "theta".
# Draw the ansath circuit using the placeholder parameter as input.

# Your code goes here:

Lets visualize how the qubit state changes for different values of the parameter $\theta$:

In [ ]:
visualize_transition(qc_roty.assign_parameters({theta:2*np.pi}))

In [ ]:
ansatz = qc_roty

## Visualize the problem landscape

We can get an intuition about our problem landscape by evaluating the expectation value of the problem, the Pauli Z matrix, with respect to our chosen ansatz, the rotational Y gate.

In [ ]:
# Depict the problem landscape by evaluating the eigenvalues for several trial states
# Define the trial states by specifying theta from 0 to 4 pi..
phases = np.linspace(0, 4*np.pi, 100)
individual_phases = [[ph] for ph in phases]

To evaluate the expacation value for the chosen trial states, we first have to choose a backend on which the simulation is run. 

Let's first choose the a local simulator that mimics the a real hardware system. This is a **classical compute resources that simulate Quantum behaviour.** 

Note:
Let's not forget to transpile to the ISA of the underlying hardware.

In [ ]:
real_backend = service.backend('ibm_brisbane')
sim_backend = AerSimulator.from_backend(real_backend)
sim_backend

In [ ]:
pm = generate_preset_pass_manager(backend=sim_backend, optimization_level=3)
isa_ansatz = pm.run(ansatz)
isa_ansatz.draw(idle_wires=False)

In [ ]:
isa_ops = ops.apply_layout(layout=isa_ansatz.layout)
isa_ops

Lets start the simulation: 

- With the 'Session' context manager we can establish a connection to the backend (and close this connection immediately after the computation is over)
- We can evaluate the eigenvalue of an observable w.r.t to a trial state, using the Qiskit-Runtime-Primitive "Estimator".

In [ ]:
# Use the session context manager to evaluate the trail wave fundtions, on our observable. (circuits,parameter_values,observables)

# Your code goes here: 


In [ ]:
plot_expect_landscape(phases,result_landscape)

Some insights from this graph: 

- The minimal eigenvalue of the Pauli Z matrix is -1. 
- The problem is ($2 \pi$) periodic.

## Solve the optimization problem with a Quantum simulator

Now its time for VQE: 

We will now use the VQE-routine to infer the minimal eigenvalue.

Remember: 

- Our objective is to find the minimal eigenvalue of the Pauli-Z matrix.
- A classical optimization algorithm will evaluate the slack parameters $\theta$ of our ansatz.

In [ ]:
# Define the VQE routine to infer the minimal expectation value
def custom_vqe_estimator(estimator,ansatz,ops,service,backend,optimizer=None,initial_point=None):
    energy_list = []
    param_list = []
    
    #Objective function is the expectation value E = <psi(theta)|OPS|psi(theta)>
    def objective_function(params):
        # The cost is the expectation value of the observable

        # Your code goes here: 

        
        print(f'Backend: {estimator.session.backend()} - Cost: {cost}')
        return cost
    
    # Call back function
    def callback(x,fx,ax,tx,nx):
    # Callback function to get a view on internal states and statistics of the optimizer for visualization
        energy_list.append(ax)
        param_list.append(fx)
            
    if initial_point is None:
        initial_point = np.random.random(ansatz.num_parameters)

    # Define optimizer and pass callback function
    if optimizer == None:
        optimizer = SPSA(maxiter=10, callback=callback)
        
       # Define minimize function
    result =  optimizer.minimize(fun=objective_function,x0=initial_point)


    return  param_list, energy_list, result           

In [ ]:
# Evaluate minimal expectation value with the VQE routine
# Check job progress on quantum-computing.ibm.com -> jobs.
with Session(service=service,backend=sim_backend) as session:
    estimator_sim = Estimator(session=session)
    param_list_sim,energy_list_sim,result_vqe = custom_vqe_estimator(estimator_sim,
                                                             isa_ansatz,
                                                             isa_ops,
                                                             service,
                                                             sim_backend,
                                                             optimizer=None,
                                                             initial_point=[1/2*np.pi])
params_sim = [p[0] for p in param_list_sim]

Let's visualize the path our optimizer was taking.

In [ ]:
plot_expect_landscape(phases,result_landscape,params_sim,energy_list_sim)

## P-set 3

1) 20 points

Considering the Pauli-Y gate as an observable:
- Choose an Ansatz (parameterized quantum circuit) that is able to capture the minimum eigenvalu of this observable.
- Write a function that takes a parameter (float) as input and returns your ansatz as parameterized quntum circuit. (Use Qiskit)

2) 20 points

Visualize the problem landscape:
- evaluate the expectation value for the Pauli-Y observable and your choice of Ansatz for various parameters.
The plot should depict the input parameters on the x-axis and the respective expectation value on the y axis.
(Use Qiskit).
- For what parameter is the expectation value minimal? Does that match your solution from p-set 2? (Answere in text.)

Hint: 100 parameters ranging from 0 to 4 $\pi$ should capture your problem landscape adequately)

3) 60 points

Choosing the Pauli-Y gate as your observable: Write your custom VQE routine to infer the minimal eigenvalue of this observable with your choice of Ansatz. (Use Qiskit)

Feel free to experiment with different optimizers and respective hyperparameters


